In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]

In [8]:
dependent=dataset[['charges']]

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
independent=sc.fit_transform(independent)
scd=StandardScaler()
dependent=scd.fit_transform(dependent)

In [10]:
independent

array([[-1.43876426, -0.45332   , -0.90861367, -1.0105187 ,  1.97058663],
       [-1.50996545,  0.5096211 , -0.07876719,  0.98959079, -0.5074631 ],
       [-0.79795355,  0.38330685,  1.58092576,  0.98959079, -0.5074631 ],
       ...,
       [-1.50996545,  1.0148781 , -0.90861367, -1.0105187 , -0.5074631 ],
       [-1.29636188, -0.79781341, -0.90861367, -1.0105187 , -0.5074631 ],
       [ 1.55168573, -0.26138796, -0.90861367, -1.0105187 ,  1.97058663]])

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

grid_param={'criterion':['squared_error','friedman_mse','absolute_error','poisson'],
            'max_features':['sqrt','log2',0], 'n_estimators':[10,100,1000]}

grid=GridSearchCV(RandomForestRegressor(),grid_param,refit=True,verbose=3,n_jobs=-1)

grid.fit(independent,dependent)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


C:\Users\DEEPAK\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
33 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\DEEPAK\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\DEEPAK\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\DEEPAK\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\DEEPAK\anaconda3\Lib\site-pac

GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_features': ['sqrt', 'log2', 0],
                         'n_estimators': [10, 100, 1000]},
             verbose=3)

In [12]:
all_param=grid.cv_results_
grid.best_params_

{'criterion': 'friedman_mse', 'max_features': 'log2', 'n_estimators': 1000}

In [13]:
import pickle
filename="FINAL_MODEL_GRIDSEARCHCV_WITHOUT_TRAIN_TEST_SPLIT_WITH_PREPROCESSING_IPOP_RF.sav"
pickle.dump(grid,open(filename, 'wb'))
pre_input=sc.transform([[1000,2000,3000,0,0]])
load_model=pickle.load(open("FINAL_MODEL_GRIDSEARCHCV_WITHOUT_TRAIN_TEST_SPLIT_WITH_PREPROCESSING_IPOP_RF.sav", 'rb'))
pre_result=load_model.predict(pre_input)
result=scd.inverse_transform([pre_result])
result

C:\Users\DEEPAK\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[18920.6393019]])

In [17]:
table=pd.DataFrame.from_dict(all_param)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.109370,0.009882,0.003125,6.250191e-03,squared_error,sqrt,10,"{'criterion': 'squared_error', 'max_features':...",0.844034,0.753853,0.843694,0.813477,0.830960,0.817204,0.033580,15
1,0.953091,0.009882,0.031249,9.488940e-07,squared_error,sqrt,100,"{'criterion': 'squared_error', 'max_features':...",0.860297,0.783896,0.862694,0.826620,0.839165,0.834534,0.028653,5
2,9.709408,0.065401,0.295930,2.228798e-02,squared_error,sqrt,1000,"{'criterion': 'squared_error', 'max_features':...",0.861159,0.781541,0.861725,0.830978,0.842904,0.835662,0.029437,3
3,0.097521,0.006057,0.009375,7.654266e-03,squared_error,log2,10,"{'criterion': 'squared_error', 'max_features':...",0.850502,0.751872,0.841769,0.802489,0.827106,0.814747,0.035397,17
4,1.000170,0.008185,0.031250,1.689915e-06,squared_error,log2,100,"{'criterion': 'squared_error', 'max_features':...",0.855950,0.777057,0.863183,0.827239,0.837960,0.832278,0.030406,10
5,9.516254,0.028016,0.281240,9.881590e-03,squared_error,log2,1000,"{'criterion': 'squared_error', 'max_features':...",0.860900,0.780300,0.862390,0.830099,0.844532,0.835644,0.030086,4
6,0.000000,0.000000,0.000000,0.000000e+00,squared_error,0,10,"{'criterion': 'squared_error', 'max_features':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
7,0.000000,0.000000,0.000000,0.000000e+00,squared_error,0,100,"{'criterion': 'squared_error', 'max_features':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
8,0.000000,0.000000,0.000000,0.000000e+00,squared_error,0,1000,"{'criterion': 'squared_error', 'max_features':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
9,0.103122,0.007654,0.000000,0.000000e+00,friedman_mse,sqrt,10,"{'criterion': 'friedman_mse', 'max_features': ...",0.844783,0.752938,0.848199,0.811569,0.814717,0.814441,0.034209,18
